In [11]:
import numpy as np
from scipy.optimize import approx_fprime, check_grad
from pickle import load

In [14]:
with open("c_err.pkl", "rb") as f:
    corr, err = load(f)

In [4]:
def generate_init(n_exps):
    # A0s = 2.0 ** -(np.arange(n_exps) + 1)
    A0s = 2.0 ** -(np.arange(n_exps - 1) + 1)
    A0s = np.append(A0s, 1 - A0s.sum())

    tau0s = np.ones(n_exps)

    return A0s, tau0s


In [37]:
def params_to_arr(params):
    n = params.size // 2

    As = np.asarray(params[:n])
    # k = np.asarray(params[n : n + 1])
    lambdas = np.asarray(params[n:])

    return As, lambdas


def sum_of_exps(t, As, lambdas):
    exp_terms = As[None, :] * np.exp(-lambdas[None, :] * t[:, None])
    return exp_terms.sum(axis=1)


def f(t, *params):
    As, lambdas = params_to_arr(params)
    return sum_of_exps(t, As, lambdas)


def loss(params, t, corr, err):
    As, lambdas = params_to_arr(params)

    # TODO: try also sum
    return np.sum((sum_of_exps(t, As, lambdas) - corr) ** 2 / err**2)


def jac(params, t, corr, err):
    As, lambdas = params_to_arr(params)

    exps = np.exp(-lambdas[None, :] * t[:, None])

    A_grads = exps.copy()
    # k_grads = np.ones_like(t)[:, None]
    lambda_grads = -As[None, :] * t[:, None] * exps

    # grads = np.concatenate([A_grads, k_grads, lambda_grads], axis=1)
    grads = np.concatenate([A_grads, lambda_grads], axis=1)

    f_t = sum_of_exps(t, As, lambdas)

    grad = 2 * grads * (f_t[:, None] - corr[:, None]) / err[:, None] ** 2

    return grad.sum(axis=0)


In [24]:
t = np.linspace(0, 1, corr.size)

A0s, tau0s = generate_init(5)
lambda0s = 1e-3 / tau0s


x0 = np.concatenate([A0s, lambda0s])



array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

In [22]:
np.isnan(err).sum()

30637

In [28]:
def select_data(t, corr):
    assert t.size == corr.size
    ind = np.argmax(corr < 0)
    print(ind)
    # mask = (t > 1) & (t < t[ind])
    mask = (t < t[ind])

    return mask


mask = select_data(t, corr)
sel_t = t[mask]
sel_corr = corr[mask]
sel_err = err[mask]

16043


In [29]:
mask.any()

True

In [40]:
approx_fprime(x0, loss, 1.4901161193847656e-08, args=(sel_t, sel_corr, sel_err))

TypeError: approx_fprime() got an unexpected keyword argument 'args'

In [35]:
jac(x0, sel_t, sel_corr, sel_err)

(16043, 5)
(16043, 5)
(16043, 10)


array([ 3.44471487e+11,  3.44471487e+11,  3.44471487e+11,  3.44471487e+11,
        3.44471487e+11, -2.47767627e+09, -1.23883814e+09, -6.19419069e+08,
       -3.09709534e+08, -3.09709534e+08])

In [34]:
g1 - g2

array([5779.93145752, 3731.93145752, 3731.93145752, 5779.93145752,
       5779.93145752, -269.69022846,  889.15488577, 1468.57744288,
        734.28872144,  734.28872144])

In [39]:
np.allclose(g1, g2)

True